In [1]:
import numpy
import theano
import theano.tensor as T
from theano import pp
x = T.dscalar('x')
y = x**2
gy = T.grad(y,x)
pp(gy)

'((fill((x ** TensorConstant{2}), TensorConstant{1.0}) * TensorConstant{2}) * (x ** (TensorConstant{2} - TensorConstant{1})))'

In [2]:
f = theano.function([x],gy)
f(4)

array(8.0)

In [4]:
x = T.dmatrix('x')
s = T.sum(1/(1+T.exp(-x)))
gs = T.grad(s,x)
pp(gs)

'(-(((-(fill((TensorConstant{1} / (TensorConstant{1} + exp((-x)))), fill(Sum{acc_dtype=float64}((TensorConstant{1} / (TensorConstant{1} + exp((-x))))), TensorConstant{1.0})) * TensorConstant{1})) / ((TensorConstant{1} + exp((-x))) * (TensorConstant{1} + exp((-x))))) * exp((-x))))'

In [12]:
x = theano.tensor.matrix()
x_specify_shape = theano.tensor.specify_shape(x,(2,2))
f = theano.function([x],(x_specify_shape**2).shape)
f([[1,3,2],[1,3,2]])

array([2, 2])

In [13]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')


[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 1.654610 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the cpu
